# Optimizacion equipo de Remo mediante Programacion Lineal Multiobjetivo

Este codigo corresponde al codigo utilizado para optimizar el equipo de remo. Hay varias celdas con información, pero se va a organizar para tener las ultimas versiones de codigo organizadas

Ejecutar el siguiente codigo para cargar las librerias!!!

In [2]:
%pip install pulp

DEPRECATION: Loading egg at /home/jota/app/anaconda/lib/python3.11/site-packages/WooCommerce-3.0.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


## Datos para el analisis

A continuacion se presentan los datos para realizar el analisis

In [8]:
import pulp


N = 13  # Remeros candidatos
P = 8   # Posiciones en la barca

# Nombre de remeros 
nombres = [
    "Rower 1",
    "Rower 2",
    "Rower 3",
    "Rower 4",
    "Rower 5",
    "Rower 6",
    "Rower 7",
    "Rower 8",
    "Rower 9",
    "Rower 10",
    "Rower 11",
    "Rower 12",
    "Rower 13" ,
    "Rower 14"
]

# Distintos parametros de cada remero
edad    = [ 55,   52 , 61 , 54, 51 , 59 , 57 , 63, 54, 52, 65, 44, 51, 57 ]
pesos = [ 94 , 71, 72 , 96 , 92 , 83, 87 , 80 , 88 , 60 , 82 , 77 , 74 , 75  ]
tecnica = [  15, 15, 12, 18, 19 , 11 , 18 , 20 , 14 , 14 , 14 , 13 , 20 , 0 ]
fuerzas = [
    205,  
    178,  
    173,  
    208,  
    160,  
    195,  
    194,  
    191,  
    190,  
    131,  
    141,  
    160,  
    150,
    183,
]

remeros = list(range(N))
posiciones = list(range(P))

# Posiciones de babor (izquierda) y estribor (derecha)
posiciones_babor = [0, 1, 2, 3]      # Posiciones 1 a 4
posiciones_estribor = [4, 5, 6, 7]   # Posiciones 5 a 8

eqPos = [ "B1", "B2" ,"B3" , "B4" , "E1" , "E2" , "E3" , "E4" ]

# Generación de posiciones preferidas para cada remero
posiciones_preferidas = [
    [4,5,0],       
    [0,3,6],       
    [2,3,5,6],     
    [1,2],         
    [1,2,5,7,3],   
    [4,5,6,7],     
    [4,5,2,1],     
    [0,1,2,3],     
    [4,5,7],       
    [2,3,6],       
    [2,3,6,5],     
    [3,7,2],       
    [0,1,2,6],     
    [1,3,5,7,6],   
]

##
# Codigo para imprimir la inormacion tabulada

campos = [nombres, edad, pesos, tecnica, fuerzas, posiciones_preferidas]
n = len(nombres)

# 2) Helper: mapea índices de posiciones a etiquetas (B1..E4)
def idxs_a_etiquetas(idxs, etiquetas):
    return [etiquetas[i] for i in idxs if 0 <= i < len(etiquetas)]

# 3) Construir filas (sin columna "Remeros")
rows = []
for i in range(n):
    pref_idx = posiciones_preferidas[i]
    pref_lbl = idxs_a_etiquetas(pref_idx, eqPos)
    rows.append([
        str(i + 1),
        str(nombres[i]),
        str(edad[i]),
        str(pesos[i]),
        str(tecnica[i]),
        str(fuerzas[i]),
        ",".join(pref_lbl),
    ])

# 4) Definir cabeceras
headers = ["#", "Nombre", "Edad", "Peso", "Técnica", "Fuerza", "Pref (pos)"]

# 5) Calcular anchos de columna dinámicos
cols = list(zip(*([headers] + rows)))
col_widths = [max(len(cell) for cell in col) + 3  for col in cols] 

# 6) Función para formatear una fila con padding
def fmt_row(cells):
    return " | ".join(cell.ljust(w) for cell, w in zip(cells, col_widths))

# 7) Imprimir tabla
line = "-+-".join("-" * w for w in col_widths)
print(fmt_row(headers))
print(line)
for r in rows:
    print(fmt_row(r))

#     | Nombre      | Edad    | Peso    | Técnica    | Fuerza    | Pref (pos)       
------+-------------+---------+---------+------------+-----------+------------------
1     | Rower 1     | 55      | 94      | 15         | 205       | E1,E2,B1         
2     | Rower 2     | 52      | 71      | 15         | 178       | B1,B4,E3         
3     | Rower 3     | 61      | 72      | 12         | 173       | B3,B4,E2,E3      
4     | Rower 4     | 54      | 96      | 18         | 208       | B2,B3            
5     | Rower 5     | 51      | 92      | 19         | 160       | B2,B3,E2,E4,B4   
6     | Rower 6     | 59      | 83      | 11         | 195       | E1,E2,E3,E4      
7     | Rower 7     | 57      | 87      | 18         | 194       | E1,E2,B3,B2      
8     | Rower 8     | 63      | 80      | 20         | 191       | B1,B2,B3,B4      
9     | Rower 9     | 54      | 88      | 14         | 190       | E1,E2,E4         
10    | Rower 10    | 52      | 60      | 14         | 131       

## Optimizacion Multiobjetivo con Enfoque Secuencial

Corresponde al codigo final para la ejecucion de la optimizacion multiobjetivo con enfoque secuencial. 

Esta version parametrizada permite  cambiar el orden de las funciones objetivo para realizacion de pruebas en cuanto a la seleccion del equipo. De esta manera se podria elegir alternativas de primera funcion de restriccion y cual es la segunda,..etc.


In [10]:
# Variables de decisión: x[i][j] = 1 si el remero i está en la posición j
x = pulp.LpVariable.dicts("x", ((i, j) for i in remeros for j in posiciones), cat='Binary')

# Diccionario para almacenar los valores óptimos de cada etapa
resultados_optimos = {}

# Funciones para cada etapa
def etapa_maximizar_fuerza(problema, resultados_optimos):
    # Función objetivo: maximizar la fuerza total
    problema += pulp.lpSum(fuerzas[i] * x[i, j] for i in remeros for j in posiciones), "Fuerza_Total"

def etapa_minimizar_peso(problema, resultados_optimos):
    # Función objetivo: minimizar el peso total
    problema += pulp.lpSum(pesos[i] * x[i, j] for i in remeros for j in posiciones), "Peso_Total"

def etapa_minimizar_diferencia_pesos(problema, resultados_optimos):
    # Variable auxiliar para la diferencia absoluta de pesos
    D = pulp.LpVariable("D", lowBound=0, cat='Continuous')

    # Función objetivo: minimizar D
    problema += D, "Diferencia_Pesos"

    # Cálculo de pesos en babor y estribor
    peso_babor = pulp.lpSum(pesos[i] * x[i, j] for i in remeros for j in posiciones_babor)
    peso_estribor = pulp.lpSum(pesos[i] * x[i, j] for i in remeros for j in posiciones_estribor)

    # Restricciones para la diferencia absoluta de pesos
    problema += peso_babor - peso_estribor <= D, "Restriccion_Diferencia1"
    problema += peso_estribor - peso_babor <= D, "Restriccion_Diferencia2"

def etapa_maximizar_edad(problema, resultados_optimos):
    # Función objetivo: maximizar la edad total
    problema += pulp.lpSum(edad[i] * x[i, j] for i in remeros for j in posiciones), "Edad_Total"

# Lista de etapas disponibles
etapas = [
    ("maximizar_fuerza", etapa_maximizar_fuerza),
    ("minimizar_peso", etapa_minimizar_peso),
    ("minimizar_diferencia_pesos", etapa_minimizar_diferencia_pesos),
    ("maximizar_edad", etapa_maximizar_edad)
]

# Función para ejecutar las etapas en el orden especificado
def ejecutar_etapas(etapas_ordenadas):
    for nombre_etapa, funcion_etapa in etapas_ordenadas:
        print(f"Ejecutando {nombre_etapa.replace('_', ' ').title()}")
        # Definimos el problema
        # Si la función objetivo es de maximización o minimización, debemos definir el problema en consecuencia
        if nombre_etapa.startswith("maximizar"):
            problema = pulp.LpProblem(nombre_etapa, pulp.LpMaximize)
        elif nombre_etapa.startswith("minimizar"):
            problema = pulp.LpProblem(nombre_etapa, pulp.LpMinimize)
        else:
            print(f"Tipo de problema desconocido para la etapa {nombre_etapa}")
            continue

        # Agregamos las restricciones comunes a todos los problemas
        # a) Cada posición debe ser ocupada por exactamente un remero
        for j in posiciones:
            problema += pulp.lpSum(x[i, j] for i in remeros) == 1, f"Posicion_{j}_Ocupada"

        # b) Cada remero puede ser asignado a como máximo una posición
        for i in remeros:
            problema += pulp.lpSum(x[i, j] for j in posiciones) <= 1, f"Remero_{i}_Una_Posicion"

        # c) Restricciones de posiciones preferidas
        for i in remeros:
            for j in posiciones:
                if j not in posiciones_preferidas[i]:
                    problema += x[i, j] == 0, f"Remero_{i}_No_Prefiere_Posicion_{j}"

        # d) Agregamos las restricciones basadas en los resultados óptimos de etapas anteriores
        for nombre_anterior, valor_optimo in resultados_optimos.items():
            if nombre_anterior == "maximizar_fuerza":
                problema += pulp.lpSum(fuerzas[i] * x[i, j] for i in remeros for j in posiciones) == valor_optimo, "Fuerza_Total_Fija"
            elif nombre_anterior == "minimizar_peso":
                problema += pulp.lpSum(pesos[i] * x[i, j] for i in remeros for j in posiciones) == valor_optimo, "Peso_Total_Fijo"
            elif nombre_anterior == "minimizar_diferencia_pesos":
                # La diferencia mínima de pesos ya está incluida en las restricciones de la etapa correspondiente
                pass
            elif nombre_anterior == "maximizar_edad":
                problema += pulp.lpSum(edad[i] * x[i, j] for i in remeros for j in posiciones) == valor_optimo, "Edad_Total_Fija"

        # Ejecutamos la función objetivo de la etapa actual
        funcion_etapa(problema, resultados_optimos)

        # Resolver el problema
        problema.solve()

        # Verificar si se encontró una solución óptima
        if problema.status != 1:
            print(f"No se encontró una solución factible en la etapa {nombre_etapa}.")
            return

        # Guardamos el valor óptimo de la etapa actual
        valor_optimo = pulp.value(problema.objective)
        resultados_optimos[nombre_etapa] = valor_optimo

        print(f"Valor Óptimo de {nombre_etapa.replace('_', ' ').title()}: {valor_optimo}\n")

    # Mostrar los resultados finales
    mostrar_resultados()

def mostrar_resultados():
    # ===============================
    # Cálculo de métricas globales
    # ===============================
    peso_babor_total = sum(pesos[i] for i in remeros for j in posiciones_babor if pulp.value(x[i, j]) == 1)
    peso_estribor_total = sum(pesos[i] for i in remeros for j in posiciones_estribor if pulp.value(x[i, j]) == 1)
    fuerza_total = sum(fuerzas[i] for i in remeros for j in posiciones if pulp.value(x[i, j]) == 1)
    edad_total = sum(edad[i] for i in remeros for j in posiciones if pulp.value(x[i, j]) == 1)
    peso_total = sum(pesos[i] for i in remeros for j in posiciones if pulp.value(x[i, j]) == 1)
    tecnica_total = sum(tecnica[i] for i in remeros for j in posiciones if pulp.value(x[i, j]) == 1)
    diferencia_pesos = abs(peso_babor_total - peso_estribor_total)

    # ===============================
    # Mostrar métricas principales
    # ===============================
    print("\n" + "=" * 60)
    print("      RESULTADOS DEL MODELO DE OPTIMIZACIÓN")
    print("=" * 60)
    print(f"Fuerza total del equipo (W): {fuerza_total:.1f}")
    print(f"Peso total del equipo (kg):  {peso_total:.1f}")
    print(f"Edad promedio del equipo:    {edad_total / 8:.2f} años")
    print(f"Técnica total del equipo:    {tecnica_total}")
    print(f"Diferencia de pesos B/E:     {diferencia_pesos:.1f} kg")
    print(f"Peso total en Babor:         {peso_babor_total:.1f} kg")
    print(f"Peso total en Estribor:      {peso_estribor_total:.1f} kg")
    print("=" * 60 + "\n")

    # ===============================
    # Tabla de asignación formateada
    # ===============================
    headers = ["Pos.", "Remero", "Lado", "Peso (kg)", "Fuerza (W)", "Edad", "Técnica"]
    rows = []

    for j in posiciones:
        for i in remeros:
            if pulp.value(x[i, j]) == 1:
                lado = "Babor" if j in posiciones_babor else "Estribor"
                etiqueta_pos = (
                    f"B{j+1}" if j in posiciones_babor else f"E{j - len(posiciones_babor) + 1}"
                )
                rows.append([
                    etiqueta_pos,
                    nombres[i],
                    lado,
                    str(pesos[i]),
                    str(fuerzas[i]),
                    str(edad[i]),
                    str(tecnica[i])
                ])

    # Calcular anchos de columna dinámicos
    cols = list(zip(*([headers] + rows)))
    col_widths = [max(len(cell) for cell in col) for col in cols]

    def fmt_row(cells):
        return " | ".join(cell.ljust(w) for cell, w in zip(cells, col_widths))

    # Imprimir tabla
    line = "-+-".join("-" * w for w in col_widths)
    print(fmt_row(headers))
    print(line)
    for r in rows:
        print(fmt_row(r))
    print("\n" + "=" * 60 + "\n")

# Ejemplo de cómo puedes cambiar el orden de las etapas:

# Orden original:
# etapas_ordenadas = [
#     ("maximizar_fuerza", etapa_maximizar_fuerza),
#     ("minimizar_peso", etapa_minimizar_peso),
#     ("minimizar_diferencia_pesos", etapa_minimizar_diferencia_pesos),
#     ("maximizar_edad", etapa_maximizar_edad)
# ]

# Cambiando el orden según lo que desees, simplemente reorganiza la lista:
etapas_ordenadas = [
    ("maximizar_fuerza", etapa_maximizar_fuerza),
    ("maximizar_edad", etapa_maximizar_edad),
    ("minimizar_peso", etapa_minimizar_peso),
    ("minimizar_diferencia_pesos", etapa_minimizar_diferencia_pesos)
]

# Ejecutamos las etapas en el orden especificado
ejecutar_etapas(etapas_ordenadas)

Ejecutando Maximizar Fuerza
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/jota/app/anaconda/lib/python3.11/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/fdfc936e17104acfb3208e0aee1f4f7a-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/fdfc936e17104acfb3208e0aee1f4f7a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 84 COLUMNS
At line 663 RHS
At line 743 BOUNDS
At line 848 ENDATA
Problem MODEL has 79 rows, 104 columns and 266 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1534 - 0.00 seconds
Cgl0002I 58 variables fixed
Cgl0004I processed model has 21 rows, 46 columns (46 integer (46 of which binary)) and 92 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -1534
Cbc0038I Before mini branch and bound, 4

## Optimizacion Multiobjetivo con Ponderacion

Corresponde al codigo final para la ejecucion de la optimizacion multiobjetivo con ponderacion

Esta version parametrizada permite  cambiar el orden de las funciones objetivo para realizacion de pruebas en cuanto a la seleccion del equipo. De esta manera se podria elegir alternativas de primera funcion de restriccion y cual es la segunda,..etc.


In [12]:
# Suponemos que las variables y datos ya han sido inicializados en la primera celda
# incluyendo: remeros, posiciones, pesos, fuerzas, edad, nombres, posiciones_preferidas,
# posiciones_babor, posiciones_estribor, eqPos, x

##
# Definir los pesos para cada función objetivo
w1 = 0.2  # Peso para la Fuerza Total (maximizar)
w2 = 0.2  # Peso para el Peso Total (minimizar)
w3 = 0.1  # Peso para la Diferencia de Pesos entre Babor y Estribor (minimizar)
w4 = 0.5  # Peso para la Edad Total (maximizar)

# Variables de decisión: x[i][j] = 1 si el remero i está en la posición j
x = pulp.LpVariable.dicts("x", ((i, j) for i in remeros for j in posiciones), cat='Binary')

# Definir el problema de optimización (maximización)
problema = pulp.LpProblem("Optimización_Multiobjetivo", pulp.LpMaximize)

# Variable auxiliar para la diferencia absoluta de pesos entre Babor y Estribor
D = pulp.LpVariable("D", lowBound=0, cat='Continuous')

# Variables auxiliares para la técnica asignada a cada posición
T = pulp.LpVariable.dicts("T", posiciones, lowBound=1, upBound=20, cat='Continuous')

# Función objetivo combinada utilizando la suma ponderada
# Para los objetivos que queremos minimizar (Peso Total y Diferencia de Pesos),
# multiplicamos por -1 para convertirlos en maximización

# Cálculo de la Fuerza Total
fuerza_total = pulp.lpSum(fuerzas[i] * x[i, j] for i in remeros for j in posiciones)

# Cálculo del Peso Total
peso_total = pulp.lpSum(pesos[i] * x[i, j] for i in remeros for j in posiciones)

# Cálculo de la Edad Total
edad_total = pulp.lpSum(edad[i] * x[i, j] for i in remeros for j in posiciones)

# Cálculo de los pesos en Babor y Estribor
peso_babor = pulp.lpSum(pesos[i] * x[i, j] for i in remeros for j in posiciones_babor)
peso_estribor = pulp.lpSum(pesos[i] * x[i, j] for i in remeros for j in posiciones_estribor)

# Función objetivo combinada
problema += (
    w1 * fuerza_total
    - w2 * peso_total       # Restamos porque queremos minimizar el peso total
    - w3 * D                # Restamos porque queremos minimizar la diferencia de pesos
    + w4 * edad_total
), "Función_Objetivo"

# Restricciones:

# a) Cada posición debe ser ocupada por exactamente un remero
for j in posiciones:
    problema += pulp.lpSum(x[i, j] for i in remeros) == 1, f"Posición_{j}_Ocupada"

# b) Cada remero puede ser asignado a como máximo una posición
for i in remeros:
    problema += pulp.lpSum(x[i, j] for j in posiciones) <= 1, f"Remero_{i}_Una_Posición"

# c) Restricciones de posiciones preferidas
for i in remeros:
    for j in posiciones:
        if j not in posiciones_preferidas[i]:
            problema += x[i, j] == 0, f"Remero_{i}_No_Prefiere_Posición_{j}"

# d) Restricciones para la diferencia absoluta de pesos entre Babor y Estribor
problema += peso_babor - peso_estribor <= D, "Restricción_Diferencia1"
problema += peso_estribor - peso_babor <= D, "Restricción_Diferencia2"

# e) Relacionar T[j] con x[i][j] y tecnica[i]
for j in posiciones:
    problema += T[j] == pulp.lpSum(tecnica[i] * x[i, j] for i in remeros), f"Definir_Tecnica_Posicion_{j}"

# f) Asegurar el orden decreciente de técnica en posiciones_ordenadas
# Definimos las posiciones ordenadas según la importancia técnica
# Por ejemplo, las posiciones más técnicas son 1, 2, 5, 6 (índices 0,1,4,5)
posiciones_ordenadas = [0, 1, 4, 5, 2, 3, 6, 7]
for k in range(len(posiciones_ordenadas) - 1):
    pos_actual = posiciones_ordenadas[k]
    pos_siguiente = posiciones_ordenadas[k + 1]
    problema += T[pos_actual] >= T[pos_siguiente], f"Orden_Tecnica_{pos_actual}_{pos_siguiente}"

# Resolver el problema
problema.solve()

# Verificar si se encontró una solución óptima
if problema.status != 1:
    print("No se encontró una solución factible.")
else:
    # ============================================
    # Mostrar resultados del modelo de optimización
    # ============================================
    print("\n" + "=" * 65)
    print("       📊  RESULTADOS DEL MODELO DE OPTIMIZACIÓN")
    print("=" * 65)

    # Cálculo de métricas globales
    fuerza_total = pulp.value(fuerza_total)
    peso_total = pulp.value(peso_total)
    edad_total = pulp.value(edad_total)
    diferencia_pesos = pulp.value(D)

    peso_babor_total = sum(
        pesos[i] for i in remeros for j in posiciones_babor if pulp.value(x[i, j]) == 1
    )
    peso_estribor_total = sum(
        pesos[i] for i in remeros for j in posiciones_estribor if pulp.value(x[i, j]) == 1
    )
    tecnica_total = sum(
        tecnica[i] for i in remeros for j in posiciones if pulp.value(x[i, j]) == 1
    )

    # Métricas generales
    print(f"Fuerza total del equipo (W): {fuerza_total:.1f}")
    print(f"Peso total del equipo (kg):  {peso_total:.1f}")
    print(f"Edad promedio (años):        {edad_total / 8:.2f}")
    print(f"Técnica total del equipo:    {tecnica_total}")
    print(f"Diferencia de pesos B/E (kg):{diferencia_pesos:.1f}")
    print(f"Peso total en Babor (kg):    {peso_babor_total:.1f}")
    print(f"Peso total en Estribor (kg): {peso_estribor_total:.1f}")
    print("=" * 65 + "\n")

    # ============================================
    # Tabla de asignación óptima
    # ============================================
    headers = ["Pos.", "Remero", "Lado", "Peso (kg)", "Fuerza (W)", "Edad", "Técnica"]
    rows = []

    for j in posiciones:
        for i in remeros:
            if pulp.value(x[i, j]) == 1:
                lado = "Babor" if j in posiciones_babor else "Estribor"
                etiqueta_pos = f"B{j+1}" if j in posiciones_babor else f"E{j - len(posiciones_babor) + 1}"
                rows.append([
                    etiqueta_pos,
                    nombres[i],
                    lado,
                    str(pesos[i]),
                    str(fuerzas[i]),
                    str(edad[i]),
                    str(tecnica[i])
                ])

    # Calcular ancho dinámico de columnas
    cols = list(zip(*([headers] + rows)))
    col_widths = [max(len(cell) for cell in col) for col in cols]

    def fmt_row(cells):
        return " | ".join(cell.ljust(w) for cell, w in zip(cells, col_widths))

    # Imprimir tabla
    line = "-+-".join("-" * w for w in col_widths)
    print(fmt_row(headers))
    print(line)
    for r in rows:
        print(fmt_row(r))
    print("\n" + "=" * 65 + "\n")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/jota/app/anaconda/lib/python3.11/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/bc23184e7d5d4fa28a5ccc8da76853be-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/bc23184e7d5d4fa28a5ccc8da76853be-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 101 COLUMNS
At line 1017 RHS
At line 1114 BOUNDS
At line 1235 ENDATA
Problem MODEL has 96 rows, 113 columns and 602 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 400.1 - 0.00 seconds
Cgl0002I 58 variables fixed
Cgl0003I 11 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 0 strengthened rows, 1 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 3 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitution